In [4]:
%load_ext autoreload
%autoreload 2

import json
import numpy as np
import pickle

#from deepfly.CameraNetwork import json2points2d

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
# Load the json file
annotations = json.load(open('deepfly3d-annotations-120821.json', 'rb'))
# Choose the last session
annotations_last_session = annotations['qt3mY5Iy6pT6EC1sjd5gx7T9kXh1']['data']

In [5]:
# Legs for each camera
left = np.array([keypoint for keypoint in range(19,34)] + [38] + [34,35,36])
front = np.array([0,1,2,3,4,7,8,9,19,20,21,22,23,26,27,28,18,37])
right = np.array([keypoint for keypoint in range(0,15)] + [18] + [15,16,17])

assert left.shape == (19,) and right.shape == (19,) and front.shape == (18,)

In [26]:
# Camera id and keypoints
camera_id_visible_joints = {
    'left_rear': (0, left),
    'left': (1, left),
    'left_front': (2, left),
    'front': (3, front),
    'right_front': (4, right),
    'right': (5, right),
    'right_rear': (6, right),
}

In [29]:
annotation_dictionary = {}
# hard-coded for now
image_folder = '/home/nely/front/DeepFly3D/prepare_data/data/210727_aJO-GAL4xUAS-CsChr/Fly001/001_Beh/behData/images/'

for folder in annotations_last_session:
    points2d = json2points2d(annotations_last_session[folder], image_shape=[1,1], num_images=10000)
    
    for (camera_id, visible_keypoints) in camera_id_visible_joints.values():
        points2d_clip = np.zeros((points2d.shape[1], 19, 2))
        # dumping all the annotations in a given order to points2d
        # order is: legs-antenna-abdomen
        if camera_id == 3:
            # for now, skip this camera
            continue
            points2d_clip[:, np.arange(visible_keypoints.size - 2)] = points2d[camera_id][:,visible_keypoints[:-2]]
            points2d_clip[:, -2] = points2d[3][:, -2] # right antenna
            points2d_clip[:, -1] = points2d[3][:, -1] # left antenna
        else:
            points2d_clip[:, np.arange(visible_keypoints.size)] = points2d[camera_id][:,visible_keypoints]
        
        for img_id in range(points2d_clip.shape[0]):
            if np.any(points2d_clip[img_id] != 0):
                image_path = image_folder + 'camera_{}_img_{:06d}.jpg'.format(camera_id, img_id)
                annotation_dictionary[image_path] = points2d_clip[img_id]


NameError: name 'json2points2d' is not defined

In [ ]:
with open('training_dict.pkl', 'wb') as f:
    pickle.dump(annotation_dictionary, f)